In [1]:
import glob
import cv2
import numpy as np
import pandas as pd
import os
import skimage.io as io
import matplotlib.image as mpimg
%matplotlib inline
from  matplotlib import pyplot as plt
import random
import os

In [2]:
df=pd.read_csv("C:/Users/ACER/Downloads/Saikat_project/Shopee/train.csv")

In [3]:
df.head()

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069


In [4]:
value_count = df.image_phash.value_counts().rename_axis('unique_values').reset_index(name='counts')

In [5]:
df['target']=np.nan

In [6]:
len(value_count['unique_values'])

28735

In [7]:
a=0
for i in value_count['unique_values']:
    b=0
    for j in df['image_phash']:
        if i==j:
            df.loc[df.index[b], 'target'] = value_count['counts'].loc[a]
        b=b+1
    a=a+1
        

In [8]:
df.title.map(
    lambda text: len(text.split(" "))
).describe()  #the product titles are not too long

count    34250.000000
mean         9.464672
std          4.481533
min          1.000000
25%          6.000000
50%          9.000000
75%         12.000000
max         61.000000
Name: title, dtype: float64

In [9]:
import re
import nltk
#nltk.download('stopwords')

In [10]:
from nltk.corpus import stopwords

In [11]:
stop = stopwords.words('english') #preprocessing steps
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower().strip()
    text = ' '.join([i if i not in stop and i.isalpha() else '' for i in text.lower().split()])
    text = ' '.join([lemmatizer.lemmatize(w) for w in word_tokenize(text)])
    text = re.sub(r"\s{2,}", " ", text)
    return text

In [12]:
df['cleaned_text']=df['title'].apply(clean_text) #preprocessing steps

In [13]:
cv_image=[]
test_folder=r'C:/Users/ACER/Downloads/Saikat_project/Shopee/train_images'

file = os.listdir(test_folder)
for j in file:

    img = cv2.imread('C:/Users/ACER/Downloads/Saikat_project/Shopee/train_images/'+j)
    img = cv2.resize(img, (75, 115))
    img = img.astype(np.float32)/255
    cv_image.append(img)

In [14]:
df1 = df.drop_duplicates(subset = ["image"])

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train_text,X_test_text,y_train_text,y_test_text=train_test_split(df1['title'], df1['target'], test_size=0.33, random_state=42)

In [17]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, load_model
from keras.layers import Embedding, LSTM, Dropout, Dense, Input, Bidirectional, Flatten, Conv2D, MaxPooling2D, concatenate, Conv1D, MaxPooling1D
import keras.backend as K
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping

In [18]:
MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = X_train_text.map(len).max()
EMBEDDING_DIM = 300
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True)
tokenizer.fit_on_texts(X_train_text.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
print('Max len:', MAX_SEQUENCE_LENGTH)

Found 20919 unique tokens.
Max len: 357


In [19]:
MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = X_train_text.map(len).max()
EMBEDDING_DIM = 300
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True)
tokenizer.fit_on_texts(X_train_text.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
print('Max len:', MAX_SEQUENCE_LENGTH)

Found 20919 unique tokens.
Max len: 357


In [20]:
MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = df1['cleaned_text'].map(len).max()
EMBEDDING_DIM = 300
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True)
tokenizer.fit_on_texts(X_train_text.values)
word_index_whole = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index_whole))
print('Max len:', MAX_SEQUENCE_LENGTH)

Found 20919 unique tokens.
Max len: 179


In [21]:
size_of_vocabulary=len(word_index_whole) + 1 #+1 for padding
print(size_of_vocabulary)

20920


In [22]:
def get_embedding_matrix(typeToLoad):
    if typeToLoad == "glove":
        EMBEDDING_FILE="C:/Users/ACER/Downloads/Saikat_project/Shopee/glove.twitter.27B.100d.txt"
        embed_size = 100    
    if typeToLoad == "glove" or typeToLoad == "fasttext":
        embeddings_index = dict()
        f = open(EMBEDDING_FILE,encoding="utf8")

        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

        f.close()
        print('Loaded %s word vectors.' % len(embeddings_index))
    

        embedding_matrix = np.zeros((size_of_vocabulary, 100))
        embeddedCount = 0
        for word, i in tokenizer.word_index.items():
            i-=1
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None: 
                embedding_matrix[i] = embedding_vector
                embeddedCount+=1
            
        
        return embedding_matrix

In [23]:
glove_embedding_matrix = get_embedding_matrix("glove")

Loaded 1193514 word vectors.


In [24]:
MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = X_train_text.map(len).max()
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True)
tokenizer.fit_on_texts(X_train_text.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
print('Max len:', MAX_SEQUENCE_LENGTH)

Found 20919 unique tokens.
Max len: 357


In [25]:
X_text_train = tokenizer.texts_to_sequences(X_train_text.values)
X_text_train = pad_sequences(X_text_train, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of train tensor:', X_text_train.shape)

Shape of train tensor: (21716, 357)


In [26]:
MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH_TEST = X_test_text.map(len).max()
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True)
tokenizer.fit_on_texts(X_test_text.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
print('Max len:', MAX_SEQUENCE_LENGTH)

Found 15051 unique tokens.
Max len: 357


In [27]:
X_text_test = tokenizer.texts_to_sequences(X_test_text.values)
X_text_test = pad_sequences(X_text_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of train tensor:', X_text_test.shape)

Shape of train tensor: (10696, 357)


In [28]:
MAX_SEQUENCE_LENGTH

357

In [29]:
X_train_image,X_test_image,y_train_image,y_test_image=train_test_split(cv_image, df1['target'], test_size=0.33, random_state=42)

In [30]:
X_img_train_array = np.asarray(X_train_image)

In [31]:
X_img_test_array = np.asarray(X_test_image)

In [32]:
df1['target']=df1['target'].astype('str')

<ipython-input-32-41aa19417421>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['target']=df1['target'].astype('str')


In [33]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
labels = mlb.fit(df1['target'].tolist())
#train_labels=mlb.transform(y_train_text.astype('category'))
#test_labels=mlb.transform(y_test_text.astype('category'))

In [34]:
train_labels=mlb.transform(y_train_text.astype('str'))

In [35]:
train_labels.shape

(21716, 11)

In [36]:
test_labels=mlb.transform(y_test_text.astype('str'))

In [37]:
test_labels.shape

(10696, 11)

In [38]:
def compile_model(embedding_matrix):
    lstm_input = Input(shape=(MAX_SEQUENCE_LENGTH,))
    x = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1], mask_zero=True, input_length=MAX_SEQUENCE_LENGTH, trainable=False)(lstm_input)
    x = Dropout(0.3)(x)
    x = LSTM(64, return_sequences = True)(x)
    x = Dropout(0.3)(x)
    x = LSTM(64)(x)
    x = Dropout(0.3)(x)
    lstm_out = Dense(18, activation = 'relu')(x)

    print(X_img_train_array.shape[1], X_img_train_array.shape[2], X_img_train_array.shape[3])
    cnn_input = Input(shape=(X_img_train_array.shape[1], X_img_train_array.shape[2], X_img_train_array.shape[3]))
    y = Conv2D(32, (3, 3), activation='relu', input_shape=(X_img_train_array.shape[1], X_img_train_array.shape[2], X_img_train_array.shape[3]))(cnn_input)
    y = MaxPooling2D(2, 2)(y)
    y = Conv2D(64, (3, 3), activation='relu')(y)
    y = MaxPooling2D(2, 2)(y)
    y = Conv2D(128, (3, 3), activation='relu')(y)
    y = MaxPooling2D(2, 2)(y)
    y = Conv2D(128, (3, 3), activation='relu')(y)
    y = MaxPooling2D(2, 2)(y)
    y = Flatten()(y)
    y = Dropout(0.3)(y)
    cnn_out = Dense(512, activation='relu')(y)

    concat_inp = concatenate([cnn_out, lstm_out])
    z = Dense(256, activation='relu')(concat_inp)
    z = Dropout(0.3)(z)
    z = Dense(128, activation='relu')(z)
    z = Dropout(0.3)(z)
    output = Dense(train_labels.shape[1], activation='sigmoid')(z)

    model = Model(inputs=[cnn_input, lstm_input], outputs=[output])
    adam = Adam(lr=0.001, decay=1e-5)
  
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [39]:
word2vec_model = compile_model(glove_embedding_matrix)
word2vec_model.summary()

115 75 3
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 115, 75, 3)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 113, 73, 32)  896         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 56, 36, 32)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 54, 34, 64)   18496       max_pooling2d[0][0]              
_____________________________________________________________________________________

In [40]:
batch_size = 64
num_epochs = 5
history=word2vec_model.fit([X_img_train_array,X_text_train],train_labels,validation_data=([X_img_test_array,X_text_test],test_labels),batch_size=batch_size, epochs=num_epochs)

Epoch 1/5
340/340 [==============================] - 292s 828ms/step - loss: 0.1691 - accuracy: 0.6996 - val_loss: 0.1212 - val_accuracy: 1.0000
Epoch 2/5
340/340 [==============================] - 275s 809ms/step - loss: 0.1248 - accuracy: 0.7338 - val_loss: 0.1195 - val_accuracy: 1.0000
Epoch 3/5
340/340 [==============================] - 277s 814ms/step - loss: 0.1231 - accuracy: 0.7329 - val_loss: 0.1194 - val_accuracy: 1.0000
Epoch 4/5
340/340 [==============================] - 11077s 33s/step - loss: 0.1191 - accuracy: 0.7682 - val_loss: 0.1196 - val_accuracy: 1.0000
Epoch 5/5
340/340 [==============================] - 245460s 724s/step - loss: 0.1194 - accuracy: 0.7743 - val_loss: 0.1182 - val_accuracy: 1.0000
